In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import copy
import japanize_matplotlib

# クラスを使わない実装

## Data

In [5]:
df = pd.read_csv("../data/df_processed.csv", index_col=0)
df.head()

,農林漁業,鉱業,飲食料品,繊維製品,パルプ・紙・木製品,化学製品,石油・石炭製品,プラスチック・ゴム製品,窯業・土石製品,鉄鋼,...,在庫純増,県内最終需要計,県内需要合計,移輸出,最終需要計,需要合計,［控除］移輸入,最終需要部門計,県内生産額,県内総支出（ＧＤＥ）
農林漁業,10518,0,35335,289,3751,201,0,104,4,0,...,2066.0,30595.0,91446.0,104718.0,135313.0,196164.0,-53219.0,82094.0,142945.0,81604.0
鉱業,3,0,40,30,20,1114,433278,5,4701,67020,...,-14907.0,-15024.0,565664.0,1926.0,-13098.0,567590.0,-564219.0,-577317.0,3371.0,-577278.0
飲食料品,4436,0,35115,62,55,7235,3,1,22,0,...,1523.0,207731.0,302802.0,139892.0,347623.0,442694.0,-240843.0,106780.0,201851.0,100491.0
繊維製品,1232,13,220,15573,364,243,13,318,85,162,...,763.0,33413.0,63689.0,60092.0,93505.0,123781.0,-58508.0,34997.0,65273.0,34200.0
パルプ・紙・木製品,5251,8,3903,333,21945,6068,7,455,512,215,...,-1049.0,5894.0,87665.0,57008.0,62902.0,144673.0,-62410.0,492.0,82263.0,-27.0


In [6]:
df.columns

Index(['農林漁業', '鉱業', '飲食料品', '繊維製品', 'パルプ・紙・木製品', '化学製品', '石油・石炭製品',
       'プラスチック・ゴム製品', '窯業・土石製品', '鉄鋼', '非鉄金属', '金属製品', 'はん用機械', '生産用機械',
       '業務用機械', '電子部品', '電気機械', '情報通信機器', '輸送機械', 'その他の製造工業製品', '建設',
       '電力・ガス・熱供給', '水道', '廃棄物処理', '商業', '金融・保険', '不動産', '運輸・郵便', '情報通信', '公務',
       '教育・研究', '医療・福祉', '他に分類されない会員制団体', '対事業所サービス', '対個人サービス', '事務用品',
       '分類不明', '内生部門計', '家計外消費支出', '民間消費支出', '一般政府消費支出', '県内総固定資本形成（公的）',
       '（民間）', '在庫純増', '県内最終需要計', '県内需要合計', '移輸出', '最終需要計', '需要合計', '［控除］移輸入',
       '最終需要部門計', '県内生産額', '県内総支出（ＧＤＥ）'],
      dtype='object')

In [7]:
df.index

Index(['農林漁業', '鉱業', '飲食料品', '繊維製品', 'パルプ・紙・木製品', '化学製品', '石油・石炭製品',
       'プラスチック・ゴム製品', '窯業・土石製品', '鉄鋼', '非鉄金属', '金属製品', 'はん用機械', '生産用機械',
       '業務用機械', '電子部品', '電気機械', '情報通信機器', '輸送機械', 'その他の製造工業製品', '建設',
       '電力・ガス・熱供給', '水道', '廃棄物処理', '商業', '金融・保険', '不動産', '運輸・郵便', '情報通信', '公務',
       '教育・研究', '医療・福祉', '他に分類されない会員制団体', '対事業所サービス', '対個人サービス', '事務用品',
       '分類不明', '内生部門計', '家計外消費支出(行)', '雇用者所得', '営業余剰', '資本減耗引当', '間接税（除関税等）',
       '［控除］経常補助金', '粗付加価値部門計', '県内生産額（ＣＴ）'],
      dtype='object')

## Analysis

In [8]:
n = 37
industries = df.index[:n]

# A: 投入係数行列
A = pd.DataFrame(np.zeros((n, n)), index=industries, columns=industries)
for row in industries:
     for col in industries:
          A.at[row, col] = df.at[row, col] / df.at[col, '県内生産額']

# Mhat: 移輸入係数行列
Mhat = pd.DataFrame(np.zeros((n, n)), index=industries, columns=industries)
for industry in industries:
     Mhat.at[industry, industry] = - df.at[industry, '［控除］移輸入'] / df.at[industry, '県内需要合計']

# L: レオンチェフ逆行列
L = np.linalg.inv(np.identity(n) - (np.identity(n) - Mhat.values) @ A.values)

# E: 移輸出ベクトル
E = df.loc[industries, '移輸出']

# c: 最終需要ベクトル
c = df.loc[industries, '県内最終需要計']

# x: 生産量ベクトル
x = L @ ((np.identity(n) - Mhat.values) @ c.values + E.values)

# レオンチェフの基本方程式を確認
x - df.loc[industries, '県内生産額']

農林漁業            -2.910383e-11
鉱業              -1.455192e-11
飲食料品             2.910383e-11
繊維製品             2.182787e-11
パルプ・紙・木製品        1.455192e-11
化学製品             1.164153e-10
石油・石炭製品         -1.164153e-10
プラスチック・ゴム製品      1.455192e-11
窯業・土石製品         -7.275958e-12
鉄鋼               0.000000e+00
非鉄金属             1.818989e-12
金属製品             0.000000e+00
はん用機械            5.820766e-11
生産用機械            2.910383e-11
業務用機械           -3.637979e-12
電子部品             1.818989e-12
電気機械            -3.637979e-12
情報通信機器          -1.364242e-12
輸送機械             1.818989e-12
その他の製造工業製品       2.182787e-11
建設               1.164153e-10
電力・ガス・熱供給        0.000000e+00
水道              -1.455192e-11
廃棄物処理           -7.275958e-12
商業               0.000000e+00
金融・保険           -2.910383e-11
不動産              1.164153e-10
運輸・郵便            0.000000e+00
情報通信             5.820766e-11
公務              -2.910383e-11
教育・研究           -5.820766e-11
医療・福祉           -1.164153e-10
他に分類されない会員制団体    7.275958e-12
対事業所サービス  

上記コードでA, x, Lは適切に定義できていることを確認

# クラスを使った実装

## Data

In [9]:
data_2011 = pd.read_excel("https://www.pref.wakayama.lg.jp/prefg/020300/sangyo/h23/toukeihyo_d/fil/t_37.xlsx",
                     index_col=0, skiprows=3)

In [10]:
def combine_columns(data):
    df = data.copy()
    combined_columns = []

    # 複数行にまたがった列名を結合
    for col in df.columns:
        combined_col = ""

        for cell in df[col].iloc[:3]:
            if isinstance(cell, str):
                combined_col += cell

        combined_columns.append(combined_col.strip())

    # 列名を更新
    df.columns = [col.replace(' ', '') for col in combined_columns]

    # 不要な3行を削除する
    df = df.iloc[3:]

    return df

df = combine_columns(data_2011)
df.index = [row[3:] for row in df.index] # 分類番号と半角スペースを落とす
df.drop(df.columns[0], axis=1, inplace=True) # 2列目を落とす
df.head()

,農林水産業,鉱業,飲食料品,繊維製品,パルプ・紙・木製品,化学製品,石油・石炭製品,プラスチック・ゴム,窯業・土石製品,鉄鋼,...,調整項(H),県内最終需要計(I)=B+～+H,県内需要合計(J)=A+I,移輸出(K),最終需要計(L)=I+K,需要合計(M)=J+K,［控除］移輸入(N),最終需要部門計(O)=L+N,県内生産額(P)=M+N,県内総支出（ＧＤＥ）(Q)=O-B
農林水産業,9882,0,36547,367,4533,166,0,614,6,0,...,41,28002,89715,103415,131417,193130,-52548,78869,140582,78455
鉱業,2,0,6,1,7,966,500938,2,3307,80674,...,1,20007,688725,1643,21650,690368,-688023,-666373,2345,-666338
飲食料品,4420,0,33839,67,82,7546,3,1,25,0,...,75,205327,292598,128303,333630,420901,-226551,107079,194350,101626
繊維製品,861,11,228,14155,278,237,17,205,58,200,...,743,30558,58355,56761,87319,115116,-53735,33584,61381,32910
パルプ・紙・木製品,5277,5,3385,384,20295,7556,9,369,238,228,...,33,4725,82186,47916,52641,130102,-58133,-5492,71969,-5937


In [11]:
# 列名の不要な部分を削除
def remove_suffix(col_name):
    # "("を含む文字列があれば、"("以降を削除して返す
    if "(" in col_name:
        return col_name.split("(")[0]
    else:
        return col_name

# 列名を変更
df.columns = [remove_suffix(col) for col in df.columns]

In [12]:
df.index

Index(['農林水産業', '鉱業', '飲食料品', '繊維製品', 'パルプ・紙・木製品', '化学製品', '石油・石炭製品',
       'プラスチック・ゴム', '窯業・土石製品', '鉄鋼', '非鉄金属', '金属製品', 'はん用機械', '生産用機械', '業務用機械',
       '電子部品', '電気機械', '情報・通信機器', '輸送機械', 'その他の製造工業製品', '建設', '電力・ガス・熱供給',
       '水道', '廃棄物処理', '商業', '金融・保険', '不動産', '運輸・郵便', '情報通信', '公務', '教育・研究',
       '医療・福祉', 'その他の非営利団体ｻｰﾋﾞｽ', '対事業所サ－ビス', '対個人サ－ビス', '事務用品', '分類不明',
       '内生部門計', '家計外消費支出(行)', '雇用者所得', '営業余剰', '資本減耗引当', '間接税（除関税等）',
       '［控除］経常補助金', '粗付加価値部門計', '県内生産額（ＣＴ）'],
      dtype='object')

In [13]:
df.columns

Index(['農林水産業', '鉱業', '飲食料品', '繊維製品', 'パルプ・紙・木製品', '化学製品', '石油・石炭製品',
       'プラスチック・ゴム', '窯業・土石製品', '鉄鋼', '非鉄金属', '金属製品', 'はん用機械', '生産用機械', '業務用機械',
       '電子部品', '電気機械', '情報・通信機器', '輸送機械', 'その他の製造工業製品', '建設', '電力・ガス・熱供給',
       '水道', '廃棄物処理', '商業', '金融・保険', '不動産', '運輸・郵便', '情報通信', '公務', '教育・研究',
       '医療・福祉', 'その他の非営利団体サービス', '対事業所サービス', '対個人サービス', '事務用品', '分類不明',
       '内生部門計', '家計外消費支出', '民間消費支出', '一般政府消費支出', '県内総固定資本形成（公的）', '（民間）',
       '在庫純増', '調整項', '県内最終需要計', '県内需要合計', '移輸出', '最終需要計', '需要合計', '［控除］移輸入',
       '最終需要部門計', '県内生産額', '県内総支出（ＧＤＥ）'],
      dtype='object')

In [14]:
df.rename(index={'その他の非営利団体ｻｰﾋﾞｽ': 'その他の非営利団体サービス',
                 '対事業所サ－ビス': '対事業所サービス',
                 '対個人サ－ビス': '対個人サービス'}, inplace=True)

## Analysis

### クラス定義

In [15]:
# main.ipynbより変更している
class InputOutputTable:
    def __init__(self, data:pd.DataFrame, n:int):
        self.data = data
        self.n = n
        self.industries = data.index[:n]
        self.A = self.calculate_A()
        self.Mhat = self.calculate_Mhat()
        self.L = self.calculate_L()
        self.G = self.calculate_G()
        self.G2 = self.calculate_G2()
        self.E = self.calculate_E()
        self.c = self.calculate_c()
        self.x = self.calculate_x()


    def calculate_A(self):
        a = pd.DataFrame(np.zeros((self.n, self.n)), index=self.industries, columns=self.industries)
        for row in self.industries:
            for col in self.industries:
                a.at[row, col] = self.data.at[row, col] / self.data.at[col, '県内生産額']

        return a


    def calculate_Mhat(self):
        mhat = pd.DataFrame(np.zeros((self.n, self.n)), index=self.industries, columns= self.industries)
        for industry in self.industries :
            mhat.at[industry , industry] = - self.data.at[industry, '［控除］移輸入'] / self.data.at[industry, '県内需要合計']

        return mhat


    def calculate_L(self):
        l = np.linalg.inv(np.identity(self.n) - (np.identity(self.n) - self.Mhat.values) @ self.A.values)

        return l


    def calculate_G(self):
        O = pd.DataFrame(np.zeros((self.n, self.n)), index=self.industries, columns= self.industries)
        for row in self.industries:
            for col in self.industries:
                O.at[row, col] = self.data.at[row, col] / self.data.at['県内生産額（ＣＴ）', row]

        g = np.linalg.inv(np.identity(self.n) - O)

        return g


    def calculate_G2(self):
        O = pd.DataFrame(np.zeros((self.n, self.n)), index=self.industries, columns= self.industries)
        for row in self.industries:
            for col in self.industries:
                O.at[row, col] = self.data.at[row, col] / self.data.at[row, '県内生産額']

        g = np.linalg.inv(np.identity(self.n) - O)

        return g


    def calculate_E(self, output:bool=True):
        e = self.data.loc[self.industries, '移輸出']

        return e


    def calculate_c(self):
        C = self.data.loc[self.industries, '県内最終需要計']

        return C


    def calculate_x(self):
        x = self.L @ ((np.identity(self.n) - self.Mhat.values) @ self.c.values + self.E.values)

        return x


    def calculate_linkage(self, option:str):
        if option == 'f': # forward linkage
            return [self.n * np.sum(self.G[i, :]) / np.sum(np.sum(self.G)) for i in range(self.n)]
        elif option == 'b': # backward linkage
            return [self.n * np.sum(self.L[:, i]) / np.sum(np.sum(self.L)) for i in range(self.n)]
        elif option == 't': # total linkage
            for i, industry in enumerate(self.industries):
                A_copy = copy.deepcopy(self.A)
                A_copy.loc[industry, :] = np.zeros(self.n)
                A_copy.loc[:, industry] = np.zeros(self.n)

                Mhat_copy = copy.deepcopy(self.Mhat)
                Mhat_copy.loc[industry, industry] = 0

                E_copy = copy.deepcopy(self.E)
                E_copy.at[industry] = 0

                c_copy = copy.deepcopy(self.c)
                c_copy.at[industry] = 0

                L_copy = np.linalg.inv(np.identity(self.n) - (np.identity(self.n) - Mhat_copy.values) @ A_copy.values)
                x_tilde = L_copy @ ((np.identity(self.n) - Mhat_copy.values) @ c_copy.values + E_copy.values)

                return x_tilde

In [16]:
IOtable_2011 = InputOutputTable(data=df, n=37)

### 各指標の算出

In [17]:
n = IOtable_2011.n
industries = IOtable_2011.industries

table = pd.DataFrame([], index=industries)
table.loc[:, '前方連関指標'] = IOtable_2011.calculate_linkage(option='f')
table.loc[:, '前方連関指標2'] = [IOtable_2011.n * np.sum(IOtable_2011.G2[i, :]) / np.sum(np.sum(IOtable_2011.G)) for i in range(IOtable_2011.n)]
table.loc[:, '後方連関指標'] = IOtable_2011.calculate_linkage(option='b')

total_x = np.sum(IOtable_2011.x)
total_linkage = np.sum(IOtable_2011.calculate_linkage(option='t'))
table['総合連関指標'] = 100* (total_x - IOtable_2011.x - total_linkage) / total_x

In [18]:
table

,前方連関指標,前方連関指標2,後方連関指標,総合連関指標
農林水産業,0.096152,0.096152,0.923559,0.455894
鉱業,31.362778,31.362778,1.082877,2.344393
飲食料品,0.089378,0.089378,0.980634,-0.278648
繊維製品,0.109171,0.109171,0.950548,1.537884
パルプ・紙・木製品,0.200580,0.200580,1.002422,1.393238
化学製品,0.131725,0.131725,1.037498,-3.524412
石油・石炭製品,0.099876,0.099876,0.775464,-8.505533
プラスチック・ゴム,0.254150,0.254150,0.955356,1.672707
窯業・土石製品,0.179448,0.179448,0.981587,1.903324
鉄鋼,0.140681,0.140681,1.516853,-12.345647


In [19]:
IOtable_2011.x.reshape(1, -1).shape

(1, 37)

In [20]:
O = pd.DataFrame(np.zeros((n, n)), index=industries, columns= industries)
for row in industries:
    for col in industries:
        O.at[row, col] = IOtable_2011.data.at[row, col] / IOtable_2011.data.at['県内生産額（ＣＴ）', row]

v = IOtable_2011.x.reshape(1, -1) @ (np.identity(n) - O)

In [21]:
v @ IOtable_2011.G

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,140582.0,2345.0,194350.0,61381.0,71969.0,431938.0,796553.0,51512.0,34631.0,1077647.0,...,299350.0,162109.0,252149.0,240671.0,488892.0,62081.0,201585.0,304577.0,8931.0,52086.0
